# Modularity and Reuse

In the tutorial *tut_02_sharks_and_fish* we created a model from scratch but we tied the parameters and initial states of the state vector into the model.
Doing this makes it hard to reuse the model and the model itself is not modular.
Constructing models into modules makes reuse much more likely and in some cases possible.

To make the sharks and fish model more reusable we want to separate out the initialisation of the parameters and states in the model.
We will start with model we finished with in *tut_02_sharks_and_fish* and make changes to modularise the model.
We will also need to separate out the units used into its own model so we can share the units across the other models.

First of all we will import the libcellml package.

In [ ]:
import libcellml

Next we will breakdown the original model into four models:
* Units model
* Parameters model
* State model
* Computiational model

Construct units model.

In [ ]:
# Construct units model.
units_model = libcellml.Model("units_model")

month = libcellml.Units("month")
month.addUnit("second", 0, 1, 2592000)
units_model.addUnits(month)

per_month = libcellml.Units("per_month")
per_month.addUnit("month", -1)
units_model.addUnits(per_month)

number_of_sharks = libcellml.Units("number_of_sharks")
units_model.addUnits(number_of_sharks)

thousands_of_fish = libcellml.Units("thousands_of_fish")
units_model.addUnits(thousands_of_fish)

per_shark_month = libcellml.Units("per_shark_month")
per_shark_month.addUnit("per_month")
per_shark_month.addUnit("number_of_sharks", -1)
units_model.addUnits(per_shark_month)


per_1000fish_month = libcellml.Units("per_1000fish_month")
per_1000fish_month.addUnit("per_month")
per_1000fish_month.addUnit("thousands_of_fish", -1)
units_model.addUnits(per_1000fish_month)


Construct parameters model.

In [ ]:
parameters_model = libcellml.Model("parameters_model")
parameters_component = libcellml.Component("parameters")
parameters_model.addComponent(parameters_component)

parameters_a = libcellml.Variable("a")
parameters_component.addVariable(parameters_a)
parameters_b = libcellml.Variable("b")
parameters_component.addVariable(parameters_b)
parameters_d = libcellml.Variable("d")
parameters_component.addVariable(parameters_d)

parameters_a.setInitialValue(-0.8)
parameters_b.setInitialValue(0.3)
parameters_d.setInitialValue(-0.6)

parameters_a.setInterfaceType("public")
parameters_b.setInterfaceType("public")
parameters_d.setInterfaceType("public")

Construct states model.

In [ ]:
states_model = libcellml.Model("states_model")
states_component = libcellml.Component("states")
states_model.addComponent(states_component)

states_sharks = libcellml.Variable("y_s")
states_component.addVariable(states_sharks)
states_fish = libcellml.Variable("y_f")
states_component.addVariable(states_fish)

states_sharks.setInitialValue(1.0)
states_fish.setInitialValue(2.0)

states_sharks.setInterfaceType("public")
states_fish.setInterfaceType("public")

Construct computational model.

In [ ]:
computational_model = libcellml.Model("sharks_v_fish")
sea_component = libcellml.Component("sea")
computational_model.addComponent(sea_component)
with open("resources/sharks_and_fish.mathml") as fh:
    mathml = fh.read()

sea_component.setMath(mathml)

time = libcellml.Variable("time")
sea_component.addVariable(time)
sharks = libcellml.Variable("y_s")
sea_component.addVariable(sharks)
fish = libcellml.Variable("y_f")
sea_component.addVariable(fish)
a = libcellml.Variable("a")
sea_component.addVariable(a)
b = libcellml.Variable("b")
sea_component.addVariable(b)
c = libcellml.Variable("c")
sea_component.addVariable(c)
d = libcellml.Variable("d")
sea_component.addVariable(d)


Only the *Units* model is a complete model, the remaining models are all missing required informatoin.
The first piece of required information is the units definitions for the *Units*, *Parameters*, and *Computational* models.
We do this by importing the units defined in the *Units* model, the libCellML *Importer* helps us with this task.

In [ ]:
units_import_source = libcellml.ImportSource()
units_import_source.setUrl('shark_v_fish_units.cellml')
units_import_source.setModel(units_model)

imported_month = libcellml.Units("month")
imported_month.setImportReference("month")
imported_month.setImportSource(units_import_source)

imported_per_month = libcellml.Units("per_month")
imported_per_month.setImportReference("per_month")
imported_per_month.setImportSource(units_import_source)

imported_number_of_sharks = libcellml.Units("number_of_sharks")
imported_number_of_sharks.setImportReference("number_of_sharks")
imported_number_of_sharks.setImportSource(units_import_source)

imported_thousands_of_fish = libcellml.Units("thousands_of_fish")
imported_thousands_of_fish.setImportReference("thousands_of_fish")
imported_thousands_of_fish.setImportSource(units_import_source)

imported_per_shark_month = libcellml.Units("per_shark_month")
imported_per_shark_month.setImportReference("per_shark_month")
imported_per_shark_month.setImportSource(units_import_source)

imported_per_1000fish_month = libcellml.Units("per_1000fish_month")
imported_per_1000fish_month.setImportReference("per_1000fish_month")
imported_per_1000fish_month.setImportSource(units_import_source)


Now we have to apply these imported units to the appropriate variable.

In [ ]:
computational_model.addUnits(imported_month)
computational_model.addUnits(imported_per_month)
computational_model.addUnits(imported_thousands_of_fish)
computational_model.addUnits(imported_number_of_sharks)
computational_model.addUnits(imported_per_shark_month)
computational_model.addUnits(imported_per_1000fish_month)

parameters_imported_per_month = imported_per_month.clone()
parameters_imported_per_shark_month = imported_per_shark_month.clone()
parameters_imported_per_1000fish_month = imported_per_1000fish_month.clone()
parameters_model.addUnits(parameters_imported_per_month)
parameters_model.addUnits(parameters_imported_per_shark_month)
parameters_model.addUnits(parameters_imported_per_1000fish_month)

states_imported_thousands_of_fish = imported_thousands_of_fish.clone()
states_imported_number_of_sharks = imported_number_of_sharks.clone()
states_model.addUnits(states_imported_thousands_of_fish)
states_model.addUnits(states_imported_number_of_sharks)

time.setUnits(imported_month)

a.setUnits(imported_per_month)
parameters_a.setUnits(parameters_imported_per_month)
b.setUnits(imported_per_shark_month)
parameters_b.setUnits(parameters_imported_per_shark_month)
c.setUnits(imported_per_month)
d.setUnits(imported_per_1000fish_month)
parameters_d.setUnits(parameters_imported_per_1000fish_month)

fish.setUnits(imported_thousands_of_fish)
states_fish.setUnits(states_imported_thousands_of_fish)
sharks.setUnits(imported_number_of_sharks)
states_sharks.setUnits(states_imported_number_of_sharks)


The last thing we need to do is import the components holding the initialised variables, and make the appropriate equivalences between the variables.
First we will import the components we need into the computational model.

In [ ]:
states_import_source = libcellml.ImportSource()
states_import_source.setUrl('shark_v_fish_states.cellml')
states_import_source.setModel(states_model)
parameters_import_source = libcellml.ImportSource()
parameters_import_source.setUrl('shark_v_fish_parameters.cellml')
parameters_import_source.setModel(parameters_model)

imported_parameters = libcellml.Component("parameters")
imported_parameters.setImportSource(parameters_import_source)
imported_parameters.setImportReference("parameters")

imported_states = libcellml.Component("states")
imported_states.setImportSource(states_import_source)
imported_states.setImportReference("states")

computational_model.addComponent(imported_parameters)
computational_model.addComponent(imported_states)



Now we can make the equivalences between the models.

In [ ]:

ghost_fish = libcellml.Variable("y_f")
imported_states.addVariable(ghost_fish)
libcellml.Variable.addEquivalence(ghost_fish, fish)

ghost_sharks = libcellml.Variable("y_s")
imported_states.addVariable(ghost_sharks)
libcellml.Variable.addEquivalence(ghost_sharks, sharks)

ghost_a = libcellml.Variable("a")
imported_parameters.addVariable(ghost_a)
libcellml.Variable.addEquivalence(ghost_a, a)

ghost_b = libcellml.Variable("b")
imported_parameters.addVariable(ghost_b)
libcellml.Variable.addEquivalence(ghost_b, b)

ghost_d = libcellml.Variable("d")
imported_parameters.addVariable(ghost_d)
libcellml.Variable.addEquivalence(ghost_d, d)

Set the appropriate interface type for the variables we need to make equivalent.

In [ ]:
a.setInterfaceType("public")
b.setInterfaceType("public")
d.setInterfaceType("public")
sharks.setInterfaceType("public")
fish.setInterfaceType("public")

Check that all imports have been resolved, if they haven't we will not get any further.

In [ ]:
if computational_model.hasUnresolvedImports():
    print("Some imports have not been properly defined.")
else:
    print("Imports are properly specified.")

With all the imports resolved we can flatten the model.
We have to flatten the model because the analyser cannot analyse a model with imports.

In [ ]:
importer = libcellml.Importer()
flattened_model = importer.flattenModel(computational_model)

Now we can follow through with solving of the model as we did in [Tutorial 02 - Sharks and Fish](./tut_02_sharks_and_fish.ipynb).

In [ ]:
analyser = libcellml.Analyser()
analyser.analyseModel(flattened_model)
for i in range(analyser.errorCount()):
    print(analyser.error(i).description())

analyser_model = analyser.model()
profile = libcellml.GeneratorProfile(libcellml.GeneratorProfile.Profile.PYTHON)
generator = libcellml.Generator()
generator.setProfile(profile)
generator.setModel(analyser_model)
code = generator.implementationCode()
# Use a utility funciton to create a module from the generated code so that we can interact with the model.
from utils.import_ext import import_module_from_string
predator_prey = import_module_from_string("predator_prey", code)
time = 0.0
step_size = 0.01
step_count = 2400
incr = int(step_count/60) + 1
model_variables = predator_prey.create_variables_array()
model_states = predator_prey.create_states_array()
model_rates = predator_prey.create_states_array()
predator_prey.initialise_variables(model_states, model_rates, model_variables)
predator_prey.compute_computed_constants(model_variables)
results = {
    "time": [None] * step_count,
}
for s in predator_prey.STATE_INFO:
    results[s["name"]] = [None] * step_count

for step in range(step_count):
    time = step * step_size
    predator_prey.compute_rates(time, model_states, model_rates, model_variables)
    
    # Update results.
    results["time"][step] = time
    for i, s in enumerate(predator_prey.STATE_INFO):
        results[s["name"]][step] = model_states[i]

    # Update states.
    model_states = [s + model_rates[i] * step_size for i, s in enumerate(model_states)]

We plot the results which should be exactly as before.

In [ ]:
import matplotlib.pyplot as plt

for s in predator_prey.STATE_INFO:
    plt.plot(results["time"], results[s["name"]], label = s["name"])
plt.legend()
plt.title("Population dynamics of shark and fish")
plt.ylabel("Population")
plt.xlabel("Time (months)")
plt.show()